In [118]:
import numpy as np
import pandas as pd
import requests
import json

In [214]:
url = "../../../datasets/Iowa_Liquor_Sales.csv"
alc_df = pd.read_csv(url).dropna()

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [215]:
def return_name(string):
  return " ".join(string.split(" ")[:-1])

pop_df= pd.read_csv("https://data.iowa.gov/resource/qtnr-zsrc.csv")
pop_df["County"] = pop_df["geographicname"].apply(return_name).str.upper()


In [216]:
def return_year(string, delimiter, index):
  return string.split(delimiter)[index]

pop_df["Year"] = pop_df["year"].apply(return_year, args=("-", 0))


In [217]:
alc_df["Year"] = alc_df["Date"].apply(return_year, args=("/", 2))
alc_df["County"] = alc_df["County"].str.upper()


In [218]:
df = pd.read_html("https://www.iowa-demographics.com/counties_by_population")[0]
df["Year"] = "2021"
df["County"] = df["County"].apply(return_name).str.upper()
df["Population"] = df["Population"]
df = df[["County", "Population", "Year"]]

In [219]:
pop_df["Population"] = pop_df["population"]
pop_df = pop_df[["County", "Population", "Year"]]
pop_df = pd.concat([pop_df, df])

In [220]:
# Combined data cleaning
def remove_last(input):
    store = input.strip().split(" ")
    if(np.char.isnumeric(store[-1])):
        return ' '.join(store[:-1])
    return ' '.join(store)

combined = alc_df.merge(pop_df, how = "left")
combined.drop(columns = ["Address", "City", "Zip Code", "Store Location", "County Number", "Invoice/Item Number", "Store Number", "Category", "Item Description", "Item Number", "Vendor Number", "Pack", "Volume Sold (Gallons)"], inplace = True)
combined = combined.dropna()
combined["Store Name"] = combined["Store Name"].apply(return_year, args=("/", 0)).apply(return_year, args=("#", 0))
combined["Store Name"] = combined["Store Name"].apply(remove_last)


In [221]:
combined

,Date,Store Name,County,Category Name,Vendor Name,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Year,Population
1,01/23/2018,Hy-Vee Food Store,BLACK HAWK,American Vodkas,SAZERAC NORTH AMERICA,1750.0,6.92,10.38,12.0,124.56,21.00,2018,131723
3,01/25/2018,Sycamore Convenience,BLACK HAWK,Flavored Rum,PERNOD RICARD USA,1000.0,10.49,15.74,36.0,566.64,36.00,2018,131723
11,01/24/2018,Hy-Vee Food and Drug,CLINTON,Canadian Whiskies,DIAGEO AMERICAS,375.0,8.00,12.00,10.0,120.00,3.75,2018,46437
22,01/24/2018,Hometown Foods,TAMA,Canadian Whiskies,CONSTELLATION BRANDS INC,1750.0,10.45,15.68,12.0,188.16,21.00,2018,16869
32,01/23/2018,Hy-Vee Food Store,BLACK HAWK,Imported Dry Gins,BACARDI USA INC,750.0,13.50,20.25,2.0,40.50,1.50,2018,131723
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19019890,10/11/2016,Wal-Mart,BLACK HAWK,Spiced Rum,DIAGEO AMERICAS,375.0,5.00,7.50,24.0,180.00,9.00,2016,133077
19019893,10/12/2016,Hy-Vee Food and Drug,CLINTON,Whiskey Liqueur,Heaven Hill Brands,500.0,4.25,6.38,3.0,19.14,1.50,2016,47248
19019896,10/13/2016,Fareway Stores,POLK,Whiskey Liqueur,SAZERAC COMPANY INC,3000.0,30.00,45.00,3.0,133.74,9.00,2016,474277
19019898,10/17/2016,Tequila's Liquor Store,POLK,100% Agave Tequila,Jim Beam Brands,750.0,20.99,31.49,6.0,188.94,4.50,2016,474277


In [223]:
from pathlib import Path  
filepath = Path('alc_dataset.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
combined.to_csv(filepath)  